In [1]:
import configparser
from datetime import datetime
import os
os.environ["JAVA_HOME"] = "C:/Program Files/Java/jdk1.8.0_333"
os.environ ['HADOOP_HOME']= "C:/Users/hoang/Downloads/hadoop-common-2.2.0-bin-master/hadoop-common-2.2.0-bin-master/bin"
import findspark as fs
fs.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql import functions as F
from pyspark.sql import types as T


import pandas as pd
pd.set_option('display.max_columns', 500)

In [2]:
config = configparser.ConfigParser()
#Normally this file should be in ~/.aws/credentials
config.read_file(open('dl.cfg'))
os.environ["AWS_ACCESS_KEY_ID"]= config['AWS']['AWS_ACCESS_KEY_ID']
os.environ["AWS_SECRET_ACCESS_KEY"]= config['AWS']['AWS_SECRET_ACCESS_KEY']


In [3]:
#AWS_ACCESS_KEY_ID=config['AWS']['AWS_ACCESS_KEY_ID']
#AWS_SECRET_ACCESS_KEY= config['AWS']['AWS_SECRET_ACCESS_KEY']
#INPUT_DATA_SD = config['AWS']['INPUT_DATA_SD']
#INPUT_DATA_LD = config['AWS']['INPUT_DATA_LD']
#OUTPUT_DATA = config['AWS']['OUTPUT_DATA']
SONG_DATA_LOCAL=config['LOCAL']['INPUT_DATA_SD_LOCAL']
LOG_DATA_LOCAL=config['LOCAL']['INPUT_DATA_LD_LOCAL']
OUTPUT_DATA_LOCAL=config['LOCAL']['OUTPUT_DATA_LOCAL']

In [7]:
spark = SparkSession.builder\
                     .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                     .getOrCreate()

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.RuntimeException: java.io.FileNotFoundException: Hadoop bin directory does not exist: C:\Users\hoang\Downloads\hadoop-common-2.2.0-bin-master\hadoop-common-2.2.0-bin-master\bin\bin -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:736)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:271)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1105)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1091)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:571)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1633)
	at org.apache.spark.SparkContext.$anonfun$new$13(SparkContext.scala:510)
	at org.apache.spark.SparkContext.$anonfun$new$13$adapted(SparkContext.scala:510)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at org.apache.spark.SparkContext.<init>(SparkCo

In [ ]:
# read data from my S3 bucket. This is the same data in workspace
songPath = 's3://udacity-dend/song_data/*/*/*/*.json'
logPath = 's3://udacity-dend/log_data/*.json'

In [ ]:
# define output paths
output = 's3a://sparkify-outputs'


## process song data
### create song_table

In [ ]:
# Step 1: Read in the song data
df_song = spark.read.json(songPath)

# check the schema
df_song.printSchema()

In [ ]:

# Step 2: extract columns to create songs table
song_cols = ['song_id', 'title', 'artist_id', 'year', 'duration']

# groupby song_id and select the first record's title in the group.
t1 = df_song.select(F.col('song_id'), 'title') \
    .groupBy('song_id') \
    .agg({'title': 'first'}) \
    .withColumnRenamed('first(title)', 'title1')

t2 = df_song.select(song_cols)
t1.toPandas().head()


In [ ]:
t2.toPandas().head()

In [ ]:
song_table_df = t1.join(t2, 'song_id') \
                .where(F.col("title1") == F.col("title")) \
                .select(song_cols)
                
song_table_df.toPandas().head()

In [ ]:
song_table_df.toPandas().shape

In [ ]:
df_song.toPandas().shape

In [ ]:
# Step 3: Write this to a parquet file
song_table_df.write.parquet('data/songs_table', partitionBy=['year', 'artist_id'], mode='Overwrite')

In [ ]:
# write this to s3 bucket
song_table_df.write.parquet(output + 'songs_table', partitionBy=['year', 'artist_id'], mode='Overwrite')

### create artists_table

In [ ]:

# define the cols
artists_cols = ["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]
df_song.printSchema()

In [ ]:
# groupby song_id and select the first record's title in the group.
t1 = df_song.select(F.col('artist_id'), 'artist_name') \
    .groupBy('artist_id') \
    .agg({'artist_name': 'first'}) \
    .withColumnRenamed('first(artist_name)', 'artist_name1')

t2 = df_song.select(artists_cols)
t1.toPandas().head()

In [ ]:
t2.toPandas().head()

In [ ]:
artists_table_df = t1.join(t2, 'artist_id') \
                .where(F.col("artist_name1") == F.col("artist_name")) \
                .select(artists_cols)
artists_table_df.toPandas().head()

In [ ]:
# write this to s3 bucket
artists_table_df.write.parquet(output + 'artists_table', mode='Overwrite')
artists_table_df.write.parquet('data/artists_table', mode='Overwrite')

In [ ]:
# read the partitioned data
df_artists_read = spark.read.option("mergeSchema", "true").parquet("data/artists_table")

## Process log data

In [ ]:
# Step 1: Read in the log data
df_log = spark.read.json(logPath)
df_log.printSchema()

In [ ]:
df_log.filter(F.col("page") == "NextSong").toPandas().head()

In [ ]:
# Step 2: filter by actions for song plays
df_log = df_log.filter(F.col("page") == "NextSong")
df_log.toPandas().shape

In [ ]:

# Step 3: extract columns for users table
users_cols = ["userId", "firstName", "lastName", "gender", "level"]
df_log.select(users_cols).toPandas().head()

In [ ]:
df_log.select(users_cols).toPandas().shape

In [ ]:
df_log.select(users_cols).dropDuplicates().toPandas().shape

In [ ]:
df_log.select(users_cols).dropDuplicates().toPandas().head()

In [ ]:

users_table_df = df_log.select(users_cols).dropDuplicates()

In [ ]:

# write this to s3 bucket
users_table_df.write.parquet(output + 'users_table', mode='Overwrite')
users_table_df.write.parquet('data/users_table', mode='Overwrite')



## Time table


In [ ]:
#create timestamp column from original timestamp column
get_timestamp = udf()
df_log.select('ts').toPandas().head()

In [ ]:

df.withColumn('epoch', f.date_format(df.epoch.cast(dataType=t.TimestampType()), "yyyy-MM-dd"))
df_log.withColumn('ts', F.date_format(df_log.ts.cast(dataType=T.TimestampType()), "yyyy-MM-dd")).toPandas().head()

In [ ]:
df_time = df_log.select('ts')
df_time.take(5)

In [ ]:

@udf
def gettimestamp(time):
    import datetime
    time = time/1000
    return datetime.datetime.fromtimestamp(time).strftime("%m-%d-%Y %H:%M:%S")

In [ ]:

df_time.withColumn("timestamp", gettimestamp("ts")).show()

In [ ]:

df_time.printSchema()

In [ ]:
get_timestamp = F.udf(lambda x: datetime.fromtimestamp( (x/1000.0) ), T.TimestampType()) 
get_hour = F.udf(lambda x: x.hour, T.IntegerType()) 
get_day = F.udf(lambda x: x.day, T.IntegerType()) 
get_week = F.udf(lambda x: x.isocalendar()[1], T.IntegerType()) 
get_month = F.udf(lambda x: x.month, T.IntegerType()) 
get_year = F.udf(lambda x: x.year, T.IntegerType()) 
get_weekday = F.udf(lambda x: x.weekday(), T.IntegerType()) 

In [ ]:
df_log = df_log.withColumn("timestamp", get_timestamp(df_log.ts))
df_log = df_log.withColumn("hour", get_hour(df_log.timestamp))
df_log = df_log.withColumn("day", get_day(df_log.timestamp))
df_log = df_log.withColumn("week", get_week(df_log.timestamp))
df_log = df_log.withColumn("month", get_month(df_log.timestamp))
df_log = df_log.withColumn("year", get_year(df_log.timestamp))
df_log = df_log.withColumn("weekday", get_weekday(df_log.timestamp))
df_log.limit(5).toPandas()

In [ ]:
time_cols = ["timestamp", "hour", "day", "week", "month", "year", "weekday"]
time_table_df = df_log.select(time_cols)

# write to parquet file partition by 
time_table_df.write.parquet('data/time_table', partitionBy=['year', 'month'], mode='Overwrite')
# write this to s3 bucket
time_table_df.write.parquet(output + 'time_table', partitionBy=['year', 'month'], mode='Overwrite')

### SongPlay table

In [ ]:
df_log.printSchema()

In [ ]:
songplay_cols_temp = ["timestamp", "userId", "sessionId", "location", "userAgent", "level"]

df_log.select(songplay_cols).toPandas().head()

In [ ]:
# read the partitioned data
df_artists_read = spark.read.option("mergeSchema", "true").parquet("data/artists_table")
df_songs_read = spark.read.option("mergeSchema", "true").parquet("data/songs_table")

df_artists_read.toPandas().head()

In [ ]:
df_songs_read.toPandas().head()

In [ ]:

# merge song and artists
df_songs_read.join(df_artists_read, 'artist_id').toPandas().head()

In [ ]:
df_joined_songs_artists = df_songs_read.join(df_artists_read, 'artist_id').select("artist_id", "song_id", "title", "artist_name")
songplay_cols = ["timestamp", "userId", "song_id", "artist_id", "sessionId", "location", "userAgent", "level", "month", "year"]

In [ ]:
# join df_logs with df_joined_songs_artists
df_log.join(df_joined_songs_artists, df_log.artist == df_joined_songs_artists.artist_name).select(songplay_cols).toPandas().head()

In [ ]:
songplay_table_df = df_log.join(df_joined_songs_artists, df_log.artist == df_joined_songs_artists.artist_name).select(songplay_cols)
songplay_table_df = songplay_table_df.withColumn("songplay_id", F.monotonically_increasing_id())
songplay_table_df.toPandas().head()

In [ ]:

# write this to parquet file
# write to parquet file partition by 
songplay_table_df.write.parquet('data/songplays_table', partitionBy=['year', 'month'], mode='Overwrite')